<a href="https://colab.research.google.com/github/ma2sevich222/raw_stock_data_researches/blob/main/trend%20prediction%20with%20optimal%20time%20embedding%20using%20complement%20NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U giotto-tda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.6/452.6 KB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.1/526.1 KB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.0 MB/s eta 0:00:00


In [ ]:
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
#from gtda.time_series import takens_embedding_optimal_parameters
import numpy as np
from sklearn.decomposition import NMF
from sklearn.naive_bayes import ComplementNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def nmf_decompose(array, n_components):
  model = NMF(n_components=n_components)
  W = model.fit_transform(array)
  H = model.components_
  return W, H



def normalize_columns(X, min_val=0, max_val=1):
    X_norm = np.zeros_like(X)
    for i in range(X.shape[1]):
        col = X[:, i]
        col_min = col.min()
        col_max = col.max()
        if col_max == col_min:
            col_norm = np.zeros_like(col)
        else:
            col_norm = (col - col_min) / (col_max - col_min)
            col_norm = min_val + (max_val - min_val) * col_norm
        X_norm[:, i] = col_norm
    return X_norm


def get_train_data(train_df, pattern_size, column_list):
  
  patterns = []
  labels = []
  for i in range(0 ,train_df.shape[0] + 1, pattern_size):
    if i - pattern_size >=0:
      sample = train_df.iloc[i - pattern_size : i]
      sample = sample.reset_index()
      labels.append(sample["Label"].iloc[-1])
      
      sample_arr = normalize_columns(sample[column_list].values)
      #_, H = nmf_decompose(sample_arr, 8)
      patterns.append(sample_arr.flatten())

  return np.array(patterns), np.array(labels)


def get_test_data(test_df,eval_df, pattern_size, column_list):

  test_patterns = []
  test_frame = pd.DataFrame(columns = ["Datetime", "Open", "High", "Low", "Close", "Volume"])
  for i in range(0 ,test_df.shape[0] + 1, pattern_size):
    if i - pattern_size >=0:
      sample = test_df.iloc[i - pattern_size : i]
      eval_data_ = eval_df.iloc[i - pattern_size : i]
      test_frame = test_frame.append(eval_data_[["Datetime", "Open", "High", "Low", "Close", "Volume"]].iloc[-1], ignore_index=True)
      sample_arr = normalize_columns(sample[column_list].values)
      #_, H = nmf_decompose(sample_arr, 8)
      test_patterns.append(sample_arr.flatten())


  return np.array(test_patterns), test_frame

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/working_dir/fixed_Apate_V4_dataset_GC_5min.csv')
#df.replace([np.inf, -np.inf], np.nan, inplace=True)
#df.dropna( how="all", inplace=True)

In [ ]:
'''count = np.isinf(df.iloc[:,1:]).values.sum()
print("It contains " + str(count) + " infinite values")'''

It contains 0 infinite values


In [ ]:
colums_list = ["Open", "High","Low", "Close", "Volume", "ema12" ,	"ema24", "macd", "signal" ,"histogram", "rsi", "momentum", "WVAP", "+band", "-band"]

In [ ]:
#colums_list = ["Open", "High","Low", "Close", "Volume"]

In [ ]:
diff_df = df.iloc[:,1:].diff()
diff_df.fillna(0, inplace=True)
diff_df["Datetime"] = df["Datetime"].values
diff_df["Label"] = df["Label"].values

In [ ]:
'''count = np.isinf(diff_df[colums_list]).values.sum()
print("It contains " + str(count) + " infinite values")'''

'count = np.isinf(diff_df[colums_list]).values.sum()\nprint("It contains " + str(count) + " infinite values")'

In [ ]:
'''for name in colums_list:
  X = diff_df[name].values[:10000]
  print(f"{name} - {takens_embedding_optimal_parameters(X, 12, 100,)}")'''

'for name in colums_list:\n  X = diff_df[name].values[:10000]\n  print(f"{name} - {takens_embedding_optimal_parameters(X, 12, 100,)}")'

In [ ]:
pattern_size = 12

In [ ]:
''' dates
2022-05-30 00:00:00
2022-06-30 00:00:00
2022-07-30 00:00:00
2022-08-24 13:40:00'''


In [ ]:
train_df = diff_df.iloc[:50000]
test_df = diff_df.iloc[50000:55000]
eval_df = df.iloc[50000:55000]

In [ ]:
#buyes_train_df = train_df[:len(train_df)//2]
#des_tr_df = train_df[len(train_df)//2:]
#buyes_train_df=buyes_train_df.reset_index(drop=True)
#des_tr_df = des_tr_df.reset_index(drop=True)
Train_X_buyes,labels_buyes = get_train_data(train_df, pattern_size, colums_list)
#Train_X_des_tr, labels_des_tr = get_train_data(des_tr_df, pattern_size, colums_list)

In [ ]:
b_clf = ComplementNB()
b_clf.fit(Train_X_buyes, labels_buyes)
#probas = b_clf.predict_proba(Train_X_des_tr)
#des_tr_clf = RandomForestClassifier(random_state=0)
#des_tr_clf.fit(probas, labels_des_tr)

ComplementNB()

In [ ]:
test_patterns, eval_frame = get_test_data(test_df, eval_df, pattern_size, colums_list)

In [ ]:

Signal = []
sell_prob = []
buy_prob = []
for test_array in test_patterns:
  pred = b_clf.predict(test_array.reshape(1, -1))
  #prob = b_clf.predict_proba(test_array.reshape(1, -1))
  #pred = des_tr_clf.predict(prob.reshape(1, -1))
  #sell_prob.append(prob[0][0])
  #buy_prob.append(prob[0][1])
  if pred ==0:
    Signal.append(-1)
  else:
    Signal.append(int(pred))

In [ ]:
eval_frame["Signal"] = Signal
eval_frame["sell_prob"] = sell_prob
eval_frame["buy_prob"] = buy_prob

In [ ]:
eval_frame

,Datetime,Open,High,Low,Close,Volume,Signal,sell_prob,buy_prob
0,2019-09-26 07:40:00,1513.8,1514.4,1513.4,1514.3,566.0,0,0.492794,0.507206
1,2019-09-26 08:40:00,1514.8,1515.1,1514.4,1514.8,612.0,-1,0.735353,0.264647
2,2019-09-26 09:40:00,1514.4,1514.5,1514.0,1514.1,228.0,-1,0.745692,0.254308
3,2019-09-26 10:40:00,1516.7,1517.3,1516.6,1516.6,1368.0,1,0.219307,0.780693
4,2019-09-26 11:40:00,1514.3,1514.3,1513.7,1514.2,723.0,1,0.583543,0.416457
...,...,...,...,...,...,...,...,...,...
411,2019-10-23 00:40:00,1492.7,1493.0,1492.5,1492.9,208.0,1,0.125510,0.874490
412,2019-10-23 01:40:00,1491.5,1491.9,1491.5,1491.7,303.0,-1,0.818192,0.181808
413,2019-10-23 02:40:00,1491.4,1491.6,1491.3,1491.5,260.0,-1,0.591192,0.408808
414,2019-10-23 03:40:00,1492.1,1492.2,1492.0,1492.2,131.0,-1,0.686396,0.313604


In [ ]:
sell = np.where(eval_frame["Signal"].values == -1)[0]
buy = np.where(eval_frame["Signal"].values == 1)[0]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x= eval_frame.index, y=eval_frame["Close"],
                    mode='lines',
                    name='close'), secondary_y=False)

fig.add_trace(go.Bar(
    x=buy,
    y=[eval_frame["Close"].max() for i in range(len(buy))],
    name='buy',
    marker_color='green', opacity=0.3
), secondary_y=False)
fig.add_trace(go.Bar(
    x=sell,
    y=[eval_frame["Close"].max() for i in range(len(sell))],
    name='sell',
    marker_color='red',opacity=0.3
),secondary_y=False)

'''fig.add_trace(go.Scatter(x= eval_frame.index, y=sell_prob,
                    mode='lines',
                    name='sell_prob'), secondary_y=True)

fig.add_trace(go.Scatter(x= eval_frame.index, y=buy_prob,
                    mode='lines',
                    name='buy_prob'), secondary_y=True)'''
fig.update_layout(
    title="Размеченное движение цены на форварде с преобразованием log > корень 3-степени")
fig.update_yaxes(range=[eval_frame["Close"].min(), eval_frame["Close"].max()], secondary_y=False)

fig.show()

In [ ]:
eval_frame.to_csv("Apate_test_check_60.csv")

In [ ]:
ex_Sig = 0
Signal = []
sell_prob = []
buy_prob = []
for test_array in test_patterns:
  pred = int(b_clf.predict(test_array.reshape(1, -1)))
 
  prob = b_clf.predict_proba(test_array.reshape(1, -1))[0]
  sell_prob.append(prob[0])
  buy_prob.append(prob[1])
  
  if prob[pred] >= 0.6:
    if pred ==0:
      ex_Sig = -1
    else:
      ex_Sig = 1
    #Signal.append(-1)
  Signal.append(ex_Sig)

In [ ]:
eval_frame["Signal"] = Signal
sell = np.where(eval_frame["Signal"].values == -1)[0]
buy = np.where(eval_frame["Signal"].values == 1)[0]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x= eval_frame.index, y=eval_frame["Close"],
                    mode='lines',
                    name='close',error_y=dict(
            type='data',
            symmetric=False,
            array=sell_prob,
            arrayminus=buy_prob)), secondary_y=False, )

fig.add_trace(go.Bar(
    x=buy,
    y=[eval_frame["Close"].max() for i in range(len(buy))],
    name='buy',
    marker_color='green', opacity=0.3
), secondary_y=False)
fig.add_trace(go.Bar(
    x=sell,
    y=[eval_frame["Close"].max() for i in range(len(sell))],
    name='sell',
    marker_color='red',opacity=0.3
),secondary_y=False)

'''fig.add_trace(go.Scatter(x= eval_frame.index, y=sell_prob,
                    mode='lines',
                    name='sell_prob'), secondary_y=True)

fig.add_trace(go.Scatter(x= eval_frame.index, y=buy_prob,
                    mode='lines',
                    name='buy_prob'), secondary_y=True)'''
fig.update_layout(
    title="Размеченное движение цены на форварде с преобразованием log > корень 3-степени")
fig.update_yaxes(range=[eval_frame["Close"].min(), eval_frame["Close"].max()], secondary_y=False)

fig.show()

In [ ]:
eval_frame

,Datetime,Open,High,Low,Close,Volume,Signal,sell_prob,buy_prob
0,2019-09-26 07:40:00,1513.8,1514.4,1513.4,1514.3,566.0,0,0.492794,0.507206
1,2019-09-26 08:40:00,1514.8,1515.1,1514.4,1514.8,612.0,-1,0.735353,0.264647
2,2019-09-26 09:40:00,1514.4,1514.5,1514.0,1514.1,228.0,-1,0.745692,0.254308
3,2019-09-26 10:40:00,1516.7,1517.3,1516.6,1516.6,1368.0,1,0.219307,0.780693
4,2019-09-26 11:40:00,1514.3,1514.3,1513.7,1514.2,723.0,1,0.583543,0.416457
...,...,...,...,...,...,...,...,...,...
411,2019-10-23 00:40:00,1492.7,1493.0,1492.5,1492.9,208.0,1,0.125510,0.874490
412,2019-10-23 01:40:00,1491.5,1491.9,1491.5,1491.7,303.0,-1,0.818192,0.181808
413,2019-10-23 02:40:00,1491.4,1491.6,1491.3,1491.5,260.0,-1,0.591192,0.408808
414,2019-10-23 03:40:00,1492.1,1492.2,1492.0,1492.2,131.0,-1,0.686396,0.313604


In [ ]:
eval_frame[eval_frame["Datetime"] == '2019-10-23 04:40:00'].index.values[0]

415

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = go.Figure()
#fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x= eval_frame.index, y=eval_frame["Close"],
                    mode='lines',
                    name='close'))


fig.add_trace(go.Scatter(
        name='sell probability',
        x=eval_frame.index,
        y=eval_frame['Close']+eval_frame['sell_prob'],
        mode='lines',
        fill='tonexty',
        fillcolor = "salmon",
        
    ))
fig.add_trace(go.Scatter(
        name='buy probability',
        x=eval_frame.index,
        y=eval_frame['Close']-eval_frame['buy_prob'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fill='tonexty',
       
      
    ))

fig.add_trace(go.Bar(
    x=buy,
    y=[eval_frame["Close"].max() for i in range(len(buy))],
    name='buy',
    marker_color='green', opacity=0.3
))
fig.add_trace(go.Bar(
    x=sell,
    y=[eval_frame["Close"].max() for i in range(len(sell))],
    name='sell',
    marker_color='red',opacity=0.3
))

'''fig.add_trace(go.Scatter(x= eval_frame.index, y=sell_prob,
                    mode='lines',
                    name='sell_prob'), secondary_y=True)

fig.add_trace(go.Scatter(x= eval_frame.index, y=buy_prob,
                    mode='lines',
                    name='buy_prob'), secondary_y=True)'''
fig.update_layout(
    title="Размеченное движение цены на форварде с преобразованием log > корень 3-степени")
fig.update_yaxes(range=[eval_frame["Close"].min(), eval_frame["Close"].max()])

fig.show()